In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Baixar o tokenizer do NLTK
nltk.download('punkt')

# Exemplo de dataset de análise de sentimentos
data = pd.read_csv("../DATA/data.csv")

df = pd.DataFrame(data)
df = df.dropna(subset=['Sentiment'])

# Mapear os sentimentos para números (Positive -> 1, Negative -> 0)
df['Sentiment'] = df['Sentiment'].map({'positive': 1, 'negative': -1, 'neutral': 0})



In [58]:
# Pré-processamento: tokenizar as sentenças
df['tokens'] = df['Sentence'].apply(lambda x: word_tokenize(x.lower()))  # Converter para minúsculas e tokenizar

# Dividir em treino e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Treinar o modelo Word2Vec nos dados de treino
w2v_model = Word2Vec(sentences=train_df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Função para converter cada review em um vetor médio de Word2Vec
def get_average_vector(tokens_list, model):
    vectors = [model.wv[word] for word in tokens_list if word in model.wv]
    if len(vectors) > 0:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

# Criar representações vetoriais médias para os textos de treino e teste
train_vectors = train_df['tokens'].apply(lambda x: get_average_vector(x, w2v_model)).tolist()
test_vectors = test_df['tokens'].apply(lambda x: get_average_vector(x, w2v_model)).tolist()



In [59]:
len(w2v_model.wv["love"])

100

In [61]:
# Converter listas de vetores em DataFrames para facilitar o uso no scikit-learn
X_train = pd.DataFrame(train_vectors)
X_test = pd.DataFrame(test_vectors)
y_train = train_df['Sentiment']
y_test = test_df['Sentiment']

# Treinar um classificador Random Forest para a análise de sentimentos
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy * 100:.2f}%")



Acurácia: 55.69%


In [63]:
# Função para inferir o sentimento de uma nova frase
def inferir_sentimento(frase, model, clf):
    # Pré-processar a frase
    tokens = word_tokenize(frase.lower())
    
    # Obter o vetor médio da frase
    vector = get_average_vector(tokens, model)
    
    # Fazer a previsão
    predicao = clf.predict([vector])
    
    # Retornar o sentimento
    if predicao == 1:
        return "Positivo"
    else:
        return "Negativo"

# Exemplo de uso da função de inferência
frase_nova = "I love these students!!!"
resultado = inferir_sentimento(frase_nova, w2v_model, clf)
print(f"Sentimento da frase: '{frase_nova}' é {resultado}")


Sentimento da frase: 'I love these students!!!' é Negativo
